**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [2]:
import pandas as pd
stats = pd.read_csv('keywords.csv')

In [4]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}
def region(row):
    for i in geo_data:
        for j in geo_data[i]:
            if j in row['keyword']:
                return i       
    return 'undefined'
stats['region'] = stats.apply(region, axis=1)
stats['region'].value_counts()



undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

In [22]:
stats.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [16]:
ratings = pd.read_table('ml-1m/ratings.dat', names = ['UserID', 'MovieID', 'Rating', 'Timestamp'], sep = '::', engine='python', header=0)
movies = pd.read_table('ml-1m/movies.dat', names = ['MovieID', 'Title', 'Genre'], sep = '::', engine='python', header=0)
joined = ratings.merge(movies, on='MovieID', how='left')
joined = joined.groupby('Title').mean().reset_index()
joined = joined.filter(items = ['Title', 'Rating'])

In [19]:
def reclass(row):
    if (row['Rating'] >= 0 and row['Rating'] <= 2):
        return 'низкий рейтинг'
    elif (row['Rating'] > 2 and row['Rating'] < 4.5):
        return 'средний рейтинг'
    elif (row['Rating'] >= 4.5 and row['Rating'] <= 5):
        return 'высокий рейтинг'
joined['reclass'] = joined.apply(reclass, axis=1)
joined['reclass'].value_counts()

средний рейтинг    3457
низкий рейтинг      219
высокий рейтинг      29
Name: reclass, dtype: int64

In [23]:
joined.head()

,Title,Rating,reclass
0,"$1,000,000 Duck (1971)",3.027027,средний рейтинг
1,'Night Mother (1986),3.371429,средний рейтинг
2,'Til There Was You (1997),2.692308,средний рейтинг
3,"'burbs, The (1989)",2.910891,средний рейтинг
4,...And Justice for All (1979),3.713568,средний рейтинг


**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [26]:
ratings_filtered = ratings.groupby('UserID').count()

In [27]:
ratings_filtered = ratings_filtered[ratings_filtered['Timestamp'] > 100].reset_index()
ratings_filtered.head()

,UserID,MovieID,Rating,Timestamp
0,2,129,129,129
1,5,198,198,198
2,8,139,139,139
3,9,106,106,106
4,10,401,401,401


In [36]:
ratings_filtered['UserID'].tolist()
ratings_fil_2 = ratings[ ratings['UserID'].isin(ratings_filtered['UserID'].tolist()) ]
ratings_grouped = ratings_fil_2.groupby('UserID').agg([min, max]).reset_index()
ratings_grouped['diff'] = ratings_grouped['Timestamp']['max'] - ratings_grouped['Timestamp']['min']
ratings_grouped['diff'].mean() / 24 / 3600

164.21127298104224

**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга